In [1]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re

import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

In [2]:
#path = "F:/U-CAN-Lymfom_A/Ashish_SampleResampling/sample_data/"

#path = "F:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB"

#source_path = "/media/andres/T7 Shield/ucan_lymfom"
source_path = "E:/U-CAN-Lymfom_A"

destination_path = os.path.join(source_path, "metadata.xlsx")
selection_dataframe = os.path.join(source_path, "FinalSelected_exams_from_U-CAN-Lymfom.xlsx" )

In [3]:
directory_list = list()
for root, dirs, files in os.walk(source_path, topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

In [4]:
remove_list = [ 'PR----BONE-PULM-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----WB-Venfas-3-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----BONE-1.25-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----Lunga-0.6-ax-mm',
                'PR----SAVED-IMAGES-PR-mm',
                'PR----e1-QCFX-S-400-Static-mm',
                'PR----WB-Venfas-0.6-ax-mm',
                'PR----WB-VEN-AX-mm',
                'PR----WB-Ben-lunga-0.6-ax-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-AX-mm',
                'PR----LUNG-1.25-AX-mm',
                'PR----THORAX-INANDAD-mm',
                'PR----KEY_IMAGES-PR-mm',
                'PR----SAVED-PR-mm',
                'Examinations that miss either CT or PET or both',
                'MR-',
                'sag',
                'cor',
                'ot-'
            ]

keep_list = ["CT-", "PT-"]

In [ ]:
findir_lst = []
rejection_lst = []

for dir in directory_list:
    dir = dir.replace('\\', '/')
    if any(item.lower() in dir.lower() for item in keep_list) and all(
            item.lower() not in dir.lower() for item in remove_list):
        print(dir)
        findir_lst.append(dir)
    else:
        rejection_lst.append(dir)

In [6]:
len(rejection_lst)

5207

In [7]:
ucan_md_dict = {'dir': [],
                'source_dir': [],
                'patient_dir': [],
                'image_dir': [],
                'dicom_img': [],
                'patient_id': [],
                'patient_age': [],
                'patient_sex': [],
                'patient_weight': [],
                'patient_size': [],
                'rows': [],
                'columns': [],
                'num_slices': [],
                'pixel_spacing': [],
                'slice_thickness': [],
                'img_pos': [],
                'img_orient': [],
                'for_uid': [],
                'att_corr': [],
                'recons_method': [],
                'image_type': [],
                'aquisition_dt': [],
                'aquisition_time': [],
                'study_desc': [],
                'series_desc': [],
                'protocol': [],
                'corr_img': [],
                'modality': [],
                'manufacturer': [],
                'manufacturer_model': [],
                'radiopharmaceutical': [],
                'radiopharmaceutical_volume': [],
                'radiopharmaceutical_start_time': [],
                'radionuclide_total_dose': [],
                'radionuclide_half_life': [],
                'radionuclide_positron_fraction': [],
                'radiopharmaceutical_start_date_time': [],
                }

#emptydir_lst = []

for dir in findir_lst:
    #if len(os.listdir(dir))<100:
    #    emptydir_lst.append(dir)
    #    continue

    ucan_md_dict['dir'].append(dir)
    ucan_md_dict['dicom_img'].append(os.listdir(dir)[0])
    
    ucan_md_dict['source_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-3])
    ucan_md_dict['patient_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-2])
    ucan_md_dict['image_dir'].append(dir.rsplit(sep='/', maxsplit=2)[-1])  
                                            
    #print(os.listdir(dir)[0])
    ds = dicom.dcmread(dir + '/' + str(os.listdir(dir)[0]))

    meta_info = str(ds.fix_meta_info)#.split('\n')
    
    max_range = 500
    
    #PatientID
    try:
        id = re.search(r"\b(0010, 0020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_id'].append(element)
    except:
        ucan_md_dict['patient_id'].append('')
    
    #PatientAge 
    try:
        id = re.search(r"\b(0010, 1010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_age'].append(element)
    except:
        ucan_md_dict['patient_age'].append('')

    #PatientSex
    try:
        id = re.search(r"\b(0010, 0040)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_sex'].append(element)
    except:
        ucan_md_dict['patient_sex'].append('')

    #Patient Size
    try:
        id = re.search(r"\b(0010, 1020)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_size'].append(element)
    except:
        ucan_md_dict['patient_size'].append('')

    #Patient Weight
    try:
        id = re.search(r"\b(0010, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['patient_weight'].append(element)
    except:
        ucan_md_dict['patient_weight'].append('')
    
    #Rows
    try:
        id = re.search(r"\b(0028, 0010)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['rows'].append(element)
    except:
        ucan_md_dict['rows'].append('')

    #Columns
    try:
        id = re.search(r"\b(0028, 0011)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['columns'].append(element)
    except:
        ucan_md_dict['columns'].append('')

    #Num of slices
    try:
        id = re.search(r"\b(07a1, 1002)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['num_slices'].append(element)
    except:
        ucan_md_dict['num_slices'].append('')

    #Pixel spacing
    try:
        id = re.search(r"\b(0028, 0030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['pixel_spacing'].append(element)
    except:
        ucan_md_dict['pixel_spacing'].append('')

    #Slice thickness
    try:
        id = re.search(r"\b(0018, 0050)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['slice_thickness'].append(element)
    except:
        ucan_md_dict['slice_thickness'].append('')

    #Image positioning
    try:
        id = re.search(r"\b(0020, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_pos'].append(element)
    except:
        ucan_md_dict['img_pos'].append('')

    #Image orientation
    try:
        id = re.search(r"\b(0020, 0037)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['img_orient'].append(element)
    except:
        ucan_md_dict['img_orient'].append('')

    #Frame of reference UID
    try:
        id = re.search(r"\b(0020, 0052)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['for_uid'].append(element)
    except:
        ucan_md_dict['for_uid'].append('')

    #Attenuation Correction Method
    try:
        id = re.search(r"\b(0054, 1101)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['att_corr'].append(element)
    except:
        ucan_md_dict['att_corr'].append('')

    #Reconstruction Method
    try:
        id = re.search(r"\b(0054, 1103)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['recons_method'].append(element)
    except:
        ucan_md_dict['recons_method'].append('')

    #Image Type
    try:
        id = re.search(r"\b(0008, 0008)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['image_type'].append(element)
    except:
        ucan_md_dict['image_type'].append('')

    #Aquisition Date
    try:
        id = re.search(r"\b(0008, 0022)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_dt'].append(element)
    except:
        ucan_md_dict['aquisition_dt'].append('')
    
    #Aquisition Time
    try:
        id = re.search(r"\b(0008, 0032)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['aquisition_time'].append(element)
    except:
        ucan_md_dict['aquisition_time'].append('')

    #Study Description
    try:
        id = re.search(r"\b(0008, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['study_desc'].append(element)
    except:
        ucan_md_dict['study_desc'].append('')

    #Series Description
    try:
        id = re.search(r"\b(0008, 103e)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['series_desc'].append(element)
    except:
        ucan_md_dict['series_desc'].append('')

    #Protocol Name 
    try:
        id = re.search(r"\b(0018, 1030)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['protocol'].append(element)
    except:
        ucan_md_dict['protocol'].append('')

    #Corrected Image
    try:
        id = re.search(r"\b(0028, 0051)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['corr_img'].append(element)
    except:
        ucan_md_dict['corr_img'].append('')

    #Modality
    try:
        id = re.search(r"\b(0008, 0060)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['modality'].append(element)
    except:
        ucan_md_dict['modality'].append('')

    #Manufacturer
    try:
        id = re.search(r"\b(0008, 0070)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer'].append(element)
    except:
        ucan_md_dict['manufacturer'].append('')

    #Manufacturer_model
    try:
        id = re.search(r"\b(0008, 1090)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['manufacturer_model'].append(element)
    except:
        ucan_md_dict['manufacturer_model'].append('')
    # Radiopharmaceutical
    try:
        id = re.search(r"\b(0018, 0031)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical'].append('')
     # Radiopharmaceutical Volume
    try:
        id = re.search(r"\b(0018, 1071)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_volume'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_volume'].append('')
     # Radiopharmaceutical Start Time
    try:
        id = re.search(r"\b(0018, 1072)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_time'].append('')
     # Radionuclide Total Dose
    try:
        id = re.search(r"\b(0018, 1074)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_total_dose'].append(element)
    except:
        ucan_md_dict['radionuclide_total_dose'].append('')
     # Radionuclide Half Life
    try:
        id = re.search(r"\b(0018, 1075)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_half_life'].append(element)
    except:
        ucan_md_dict['radionuclide_half_life'].append('')
     # Radionuclide Positron Fraction
    try:
        id = re.search(r"\b(0018, 1076)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radionuclide_positron_fraction'].append(element)
    except:
        ucan_md_dict['radionuclide_positron_fraction'].append('')
     # Radiopharmaceutical Start Date Time
    try:
        id = re.search(r"\b(0018, 1078)\b", meta_info).start()
        element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
        ucan_md_dict['radiopharmaceutical_start_date_time'].append(element)
    except:
        ucan_md_dict['radiopharmaceutical_start_date_time'].append('')


c:\Users\asch4890\AppData\Local\anaconda3\envs\dlfia\lib\site-packages\pydicom\dataelem.py:868: UserWarning: Expected total bytes to be an even multiple of bytes per value. Instead received b'\x00\x00' with length 2 and struct format 'l' which corresponds to bytes per value of 4. This occurred while trying to parse (0009, 109d) according to VR 'SL'. Setting VR to 'UN'.
  warnings.warn(f"{message} Setting VR to 'UN'.")
c:\Users\asch4890\AppData\Local\anaconda3\envs\dlfia\lib\site-packages\pydicom\dataelem.py:868: UserWarning: Expected total bytes to be an even multiple of bytes per value. Instead received b'\x00\x00' with length 2 and struct format 'l' which corresponds to bytes per value of 4. This occurred while trying to parse (0009, 10b5) according to VR 'SL'. Setting VR to 'UN'.
  warnings.warn(f"{message} Setting VR to 'UN'.")
c:\Users\asch4890\AppData\Local\anaconda3\envs\dlfia\lib\site-packages\pydicom\dataelem.py:868: UserWarning: Expected total bytes to be an even multiple of 

In [ ]:
id = re.search(r"\b(0010, 1010)\b", meta_info).start()
element = meta_info[id:id+max_range].split('\n')[0].split(':')[1]
ucan_md_dict['patient_age'].append(element)

In [8]:
"""
with open(r'F:/U-CAN-Lymfom_A/Ashish_SampleResampling/metadata_extraction/dir_lessthan_100images.txt', 'w') as fp:
    for item in emptydir_lst:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

len(emptydir_lst)

Done
2399
"""

'\nwith open(r\'F:/U-CAN-Lymfom_A/Ashish_SampleResampling/metadata_extraction/dir_lessthan_100images.txt\', \'w\') as fp:\n    for item in emptydir_lst:\n        # write each item on a new line\n        fp.write("%s\n" % item)\n    print(\'Done\')\n\nlen(emptydir_lst)\n\nDone\n2399\n'

In [9]:
ucan_md = pd.DataFrame(ucan_md_dict)
print(ucan_md.shape)
#(8325, 29)
#(8275, 29)
ucan_md.head(2)

(8267, 37)


,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,num_slices,pixel_spacing,slice_thickness,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,1,"[0.545455, 0.596847]",'1250.090942',"[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','124149','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,1.2.840.113619.2.5.1207072.11651.1470306878.205,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,1153,"[0.976562, 0.976562]",'0.949219',"[-233.886719, -207.519531, 134.462494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20160805','130733','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,


In [10]:
ucan_md.dtypes
ucan_md['imgsz_x'] = np.int64(ucan_md['rows'].str.replace("'","").replace(" ",""))
ucan_md['imgsz_y'] = np.int64(ucan_md['columns'])
ucan_md['num_slices'] = np.int64(ucan_md['num_slices'])
ucan_md['voxsz_x'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[0])))
ucan_md['voxsz_y'] = ucan_md['pixel_spacing'].apply(lambda x: (np.float64(x.replace(" ","").replace("[","").replace("]","").split(',')[1])))
ucan_md['slice_thickness'] = np.float64(ucan_md['slice_thickness'].str.replace("'","").replace(" ",""))
ucan_md.head(2)

,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,num_slices,pixel_spacing,slice_thickness,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time,imgsz_x,imgsz_y,voxsz_x,voxsz_y
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,1,"[0.545455, 0.596847]",1250.090942,"[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','124149','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,,2291,888,0.545455,0.596847
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,1.2.840.113619.2.5.1207072.11651.1470306878.205,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,1153,"[0.976562, 0.976562]",0.949219,"[-233.886719, -207.519531, 134.462494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20160805','130733','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,,512,512,0.976562,0.976562


In [11]:
ucan_md['image_size'] = ucan_md.apply(lambda x: (x.imgsz_x, x.imgsz_y, x.num_slices), axis=1)
ucan_md['voxel_size'] = ucan_md.apply(lambda x: (x.voxsz_x, x.voxsz_y, x.slice_thickness), axis=1)
ucan_md.head(2)

,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,num_slices,pixel_spacing,slice_thickness,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time,imgsz_x,imgsz_y,voxsz_x,voxsz_y,image_size,voxel_size
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,1,"[0.545455, 0.596847]",1250.090942,"[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','124149','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,,2291,888,0.545455,0.596847,"(2291, 888, 1)","(0.545455, 0.596847, 1250.090942)"
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,1.2.840.113619.2.5.1207072.11651.1470306878.205,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,1153,"[0.976562, 0.976562]",0.949219,"[-233.886719, -207.519531, 134.462494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20160805','130733','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,,512,512,0.976562,0.976562,"(512, 512, 1153)","(0.976562, 0.976562, 0.949219)"


In [12]:
ucan_md.columns

Index(['dir', 'source_dir', 'patient_dir', 'image_dir', 'dicom_img',
       'patient_id', 'patient_age', 'patient_sex', 'patient_weight',
       'patient_size', 'rows', 'columns', 'num_slices', 'pixel_spacing',
       'slice_thickness', 'img_pos', 'img_orient', 'for_uid', 'att_corr',
       'recons_method', 'image_type', 'aquisition_dt', 'aquisition_time',
       'study_desc', 'series_desc', 'protocol', 'corr_img', 'modality',
       'manufacturer', 'manufacturer_model', 'radiopharmaceutical',
       'radiopharmaceutical_volume', 'radiopharmaceutical_start_time',
       'radionuclide_total_dose', 'radionuclide_half_life',
       'radionuclide_positron_fraction', 'radiopharmaceutical_start_date_time',
       'imgsz_x', 'imgsz_y', 'voxsz_x', 'voxsz_y', 'image_size', 'voxel_size'],
      dtype='object')

In [13]:
#order columns
new_col_lst = ['dir', 'source_dir', 'patient_dir', 'image_dir', 'dicom_img',
       'patient_id', 'patient_age', 'patient_sex', 'patient_weight',
       'patient_size', 'rows', 'columns', 'imgsz_x', 'imgsz_y', 'num_slices', 
       'voxsz_x', 'voxsz_y', 'slice_thickness',  'pixel_spacing',
       'image_size', 'voxel_size', 'img_pos', 'img_orient', 'for_uid', 'att_corr',
       'recons_method', 'image_type', 'aquisition_dt', 'aquisition_time', 'study_desc',
       'series_desc', 'protocol', 'corr_img', 'modality', 'manufacturer',
       'manufacturer_model', 'radiopharmaceutical', 'radiopharmaceutical_volume',
       'radiopharmaceutical_start_time', 'radionuclide_total_dose', 
       'radionuclide_half_life', 'radionuclide_positron_fraction', 
       'radiopharmaceutical_start_date_time'
       ]

ucan_md = ucan_md[new_col_lst]

In [14]:
print("Toal CT folders: ", ucan_md[ucan_md['image_dir'].str.contains('CT')].shape[0])
print("Toal PT folders: ", ucan_md[ucan_md['image_dir'].str.contains('PT')].shape[0])

Toal CT folders:  6634
Toal PT folders:  2708


In [15]:
ucan_md.to_excel(destination_path)

In [16]:
ucan_md.num_slices.min()

1

In [17]:
ucan_md[ucan_md['num_slices']<=100].to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices.xlsx'))

lessthan100slices_summ = ucan_md[np.int64(ucan_md['num_slices'])<=100].groupby(['modality', 'slice_thickness', 'num_slices']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
lessthan100slices_summ.to_excel(os.path.join(source_path, 'exams_with_less_than_100_slices_final.xlsx'))

print(lessthan100slices_summ.shape)
#(785, 4)
lessthan100slices_summ.head(10)

(782, 4)


,modality,slice_thickness,num_slices,dicom_img
0,'CT',1250.090942,1,308
1,'CT',1250.090942,2,65
2,'CT',1310.818237,2,20
3,'CT',1100.636353,2,19
4,'CT',1.250000,17,6
5,'CT',957.545471,2,4
6,'CT',1018.545471,2,4
7,'CT',1100.727339,2,4
8,'CT',900.090881,1,3
9,'CT',988.636414,2,3


In [18]:
ucan_md_voxeldist = ucan_md.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_md_voxeldist.xlsx'))
ucan_md_voxeldist.head(10)


,modality,voxel_size,dicom_img
0,'PT',"(1.953125, 1.953125, 2.8)",740
1,'PT',"(3.90625, 3.90625, 3.27)",738
2,'CT',"(0.976562, 0.976562, 0.625)",630
3,'PT',"(1.953125, 1.953125, 2.79)",527
4,'CT',"(0.545455, 0.596847, 1250.090942)",373
5,'PT',"(3.90625, 3.90625, 2.8)",355
6,'PT',"(3.90625, 3.90625, 2.79)",275
7,'CT',"(0.976562, 0.976562, 3.0)",220
8,'CT',"(0.703125, 0.703125, 1.25)",166
9,'CT',"(1.367188, 1.367188, 3.75)",156


In [19]:
ucan_md_imgszdist = ucan_md.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_md_imgszdist.xlsx'))
ucan_md_imgszdist.head(10)

,modality,image_size,dicom_img
0,'PT',"(128, 128, 247)",478
1,'PT',"(256, 256, 314)",451
2,'PT',"(128, 128, 287)",318
3,'CT',"(2291, 888, 1)",308
4,'CT',"(512, 512, 322)",293
5,'PT',"(256, 256, 287)",239
6,'PT',"(128, 128, 314)",237
7,'CT',"(512, 512, 1151)",232
8,'CT',"(512, 512, 354)",230
9,'CT',"(512, 512, 644)",192


In [20]:
ucan_md_gt200slices = ucan_md[ucan_md['num_slices']>=200]
ucan_md_gt200slices.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices.xlsx'))
print(ucan_md_gt200slices.shape)
ucan_md_gt200slices.head(2)

(6522, 43)


,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,1.2.840.113619.2.5.1207072.11651.1470306878.205,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,512,512,1153,0.976562,0.976562,0.949219,"[0.976562, 0.976562]","(512, 512, 1153)","(0.976562, 0.976562, 0.949219)","[-233.886719, -207.519531, 134.462494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20160805','130733','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,
2,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-130439-6.3-UAS-WB-FDG-3D-CT-VENFAS-LUNG-1.25-AX-1.250000mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-130439-6.3-UAS-WB-FDG-3D-CT-VENFAS-LUNG-1.25-AX-1.250000mm,1.2.840.113619.2.55.3.4240671514.785.1470288680.829.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,512,512,230,0.976562,0.976562,1.250000,"[0.976562, 0.976562]","(512, 512, 230)","(0.976562, 0.976562, 1.25)","[-250.000, -250.000, 56.500]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20160805','130733','PET FDG WB 1 h','LUNG 1.25 AX','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,


In [21]:
ucan_md_gt200slices.groupby('voxel_size').agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False)

,dicom_img
voxel_size,
"(1.953125, 1.953125, 2.8)",736
"(3.90625, 3.90625, 3.27)",732
"(0.976562, 0.976562, 0.625)",630
"(1.953125, 1.953125, 2.79)",519
"(3.90625, 3.90625, 2.8)",353
...,...
"(1.056641, 1.056641, 0.625)",1
"(1.054688, 1.054688, 3.75)",1
"(0.746094, 0.746094, 0.746094)",1


In [22]:
ucan_md_gt200slices_voxeldist = ucan_md_gt200slices.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_voxeldist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_voxeldist.xlsx'))
ucan_md_gt200slices_voxeldist.head(10)


,modality,voxel_size,dicom_img
0,'PT',"(1.953125, 1.953125, 2.8)",736
1,'PT',"(3.90625, 3.90625, 3.27)",732
2,'CT',"(0.976562, 0.976562, 0.625)",630
3,'PT',"(1.953125, 1.953125, 2.79)",519
4,'PT',"(3.90625, 3.90625, 2.8)",353
5,'PT',"(3.90625, 3.90625, 2.79)",272
6,'CT',"(0.976562, 0.976562, 3.0)",217
7,'CT',"(0.703125, 0.703125, 1.25)",158
8,'CT',"(1.367188, 1.367188, 3.75)",153
9,'CT',"(0.976562, 0.976562, 3.75)",85


In [23]:
ucan_md_gt200slices_imgszdist = ucan_md_gt200slices.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
ucan_md_gt200slices_imgszdist.to_excel(os.path.join(source_path, 'ucan_exams_with_more_than_200_slices_imagesizedist.xlsx'))
ucan_md_gt200slices_imgszdist.head(10)

,modality,image_size,dicom_img
0,'PT',"(128, 128, 247)",478
1,'PT',"(256, 256, 314)",451
2,'PT',"(128, 128, 287)",318
3,'CT',"(512, 512, 322)",293
4,'PT',"(256, 256, 287)",239
5,'PT',"(128, 128, 314)",237
6,'CT',"(512, 512, 1151)",232
7,'CT',"(512, 512, 354)",230
8,'CT',"(512, 512, 644)",192
9,'CT',"(512, 512, 382)",184


In [24]:
Lessthan_200Slices = ucan_md[ucan_md['num_slices']<200]
Lessthan_200Slices.to_excel(os.path.join(source_path, "less_than_200_slices.xlsx"))
print(Lessthan_200Slices.shape)
Lessthan_200Slices.head(2)

(1745, 43)


,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,2291,888,1,0.545455,0.596847,1250.090942,"[0.545455, 0.596847]","(2291, 888, 1)","(0.545455, 0.596847, 1250.090942)","[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,,,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','124149','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery STE',,,,,,,
7,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr462153436708-20220302/CT-20220302-130340-3.8-WB-F18-FDG-VENFAS-SCOUT-957.727295mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr462153436708-20220302,CT-20220302-130340-3.8-WB-F18-FDG-VENFAS-SCOUT-957.727295mm,1.2.840.113619.2.405.3.3842664208.13.1646146733.689.2,'ASPTCTX0001_npr462153436708','076Y','O','75.0','1.68',1755,835,1755,835,2,0.545455,0.634731,957.727295,"[0.545455, 0.634731]","(1755, 835, 2)","(0.545455, 0.634731, 957.727295)","[0.000, 265.000, 10.000]","[0.000000, -1.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.405.3.3842664208.13.1646146733.682.6715.1,,,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20220302','130433.876454','PET FDG WB 1 h','SCOUT','3.8 WB F18 FDG VENFAS',,'CT','GE MEDICAL SYSTEMS','Discovery MI',,,,,,,


In [25]:
ucan_md[ucan_md['image_dir'].str.contains('-ax-')][ucan_md['num_slices']<200][ucan_md['image_dir'].str.contains('PT-')]

C:\Users\asch4890\AppData\Local\Temp\ipykernel_21108\1214850953.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ucan_md[ucan_md['image_dir'].str.contains('-ax-')][ucan_md['num_slices']<200][ucan_md['image_dir'].str.contains('PT-')]


,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time


### Match metadata with selected files

In [3]:
metadata = pd.read_excel(destination_path)
# metadata['dir'] = metadata['dir'].str.replace('/','\\')
print(metadata.shape)
metadata.head(2)

(8267, 44)


,Unnamed: 0,dir,source_dir,patient_dir,image_dir,dicom_img,patient_id,patient_age,patient_sex,patient_weight,patient_size,rows,columns,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,pixel_spacing,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-124130-6.3-UAS-WB-FDG-3D-CT-VENFAS-CT-SCOUT-HEAD-IN-1250.090942mm,1.2.840.113619.2.55.3.4240671514.785.1470288679.289.1,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',2291,888,2291,888,1,0.545455,0.596847,1250.090942,"[0.545455, 0.596847]","(2291, 888, 1)","(0.545455, 0.596847, 1250.090942)","[-265.000, 0.000, 350.000]","[1.000000, -0.000000, 0.000000, 0.000000, 0.000000, -1.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'LOCALIZER']",'20160805','124149','PET FDG WB 1 h','CT SCOUT HEAD IN','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB/ASPTCTX0001_npr258071002437-20160805/CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr258071002437-20160805,CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.949219mm_JK_,1.2.840.113619.2.5.1207072.11651.1470306878.205,'ASPTCTX0001_npr258071002437','070Y','O','46.0','1.56',512,512,512,512,1153,0.976562,0.976562,0.949219,"[0.976562, 0.976562]","(512, 512, 1153)","(0.976562, 0.976562, 0.949219)","[-233.886719, -207.519531, 134.462494]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.785.1470288679.287.12920.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20160805','130733','PET FDG WB 1 h','WB VEN AX','6.3 UAS-WB-FDG-3D CT VENFAS',NaN,'CT','GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
selected_imgs = pd.read_excel(selection_dataframe)
print(selected_imgs.shape)
selected_imgs.head(2)

(1336, 8)


,Unnamed: 0,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
0,0,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc/ASPTCTX0001_npr748236341307-20220908/PT-20220908-103801-3.11-WB-F18-FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc,ASPTCTX0001_npr748236341307-20220908,PT-20220908-103801-3.11-WB-F18-FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm,ASPTCTX0001,npr748236341307,20220908
1,1,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc/ASPTCTX0001_npr748236341307-20220908/CT-20220908-104539-3.11-WB-F18-FDG-VENFAS--THORAX-INANDAD-WB-Venfas-0.6-ax-0.625000mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc,ASPTCTX0001_npr748236341307-20220908,CT-20220908-104539-3.11-WB-F18-FDG-VENFAS--THORAX-INANDAD-WB-Venfas-0.6-ax-0.625000mm,ASPTCTX0001,npr748236341307,20220908


In [5]:
master_data = pd.merge(selected_imgs, metadata, how="inner", left_on=['patient_directory', 'PET-CT_info'], right_on=['patient_dir', 'image_dir'], sort=True, suffixes=("_x", "_y"))

col_drop_lst = ['dir', 'source_dir', 'patient_dir',	'image_dir', 'patient_sex', 'rows',	'columns', 'pixel_spacing']

master_data = master_data.drop(columns=col_drop_lst)

master_data['patient_id'] = master_data['patient_id'].apply(lambda x: x.split('_')[1].replace("'",""))
master_data['patient_age'] = master_data['patient_age'].apply(lambda x: x.replace("'","").replace("Y","").replace(" ",""))#.astype('Int64')
master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())

master_data.to_excel(os.path.join(source_path, "Finalized_dataset.xlsx"))

print(master_data.shape)
master_data.head(2)

(1340, 44)


,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,1319,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc/ASPTCTX0001_npr100169878450-20130412/CT-20130412-103438-6.1-UAS-WB-FDG-3D-STANDARD-CT-RECON-3.750000mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr100169878450-20130412,CT-20130412-103438-6.1-UAS-WB-FDG-3D-STANDARD-CT-RECON-3.750000mm,ASPTCTX0001,npr100169878450,20130412,5717,1.2.840.113619.2.55.3.4240671514.401.1365057164.823.1,npr100169878450,068,'91.0','1.91',512,512,250,0.976562,0.976562,3.75,"(512, 512, 250)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 196.500]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.401.1365057164.532.13330.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20130412','103452','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1318,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc/ASPTCTX0001_npr100169878450-20130412/PT-20130412-103617-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc,ASPTCTX0001_npr100169878450-20130412,PT-20130412-103617-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,npr100169878450,20130412,5718,1.2.840.113619.2.131.4240671514.1365756118.268716,npr100169878450,068,'91.0','1.91',128,128,287,3.906250,3.906250,3.27,"(128, 128, 287)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, -738.70001220703]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.401.1365057164.532.13330.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20130412','103617','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','093300.00','347633344.0','6588.0','0.97000002861023','20130412093300.00'


In [8]:
print(master_data[master_data['patient_age']==''].shape)

master_data[master_data['patient_age']==''].head(2)

(4, 44)


,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
939,1091,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc/ASPTCTX0001_npr671684118786-20160518/PT-20160518-133807-UAS-WB-FDG-3D-VENFASTHORAX-INSP-20160120-WB-FDG-AC-3.2700mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215__17_n199_424GBondisc,ASPTCTX0001_npr671684118786-20160518,PT-20160518-133807-UAS-WB-FDG-3D-VENFASTHORAX-INSP-20160120-WB-FDG-AC-3.2700mm,ASPTCTX0001,npr671684118786,20160518,6625,1.2.840.113619.2.131.4240671514.1463571833.840715,npr671684118786,,'49.0','1.52',128,128,248,3.90625,3.90625,3.27,"(128, 128, 248)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, -695.40002441406]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.187.1463121243.236.11980.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20160518','133807','PET FDG WB 1 h','WB-FDG-AC','UAS-WB-FDG-3D VENFAS+THORAX INSP 20160120',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','123900.00','235495952.0','6588.0','0.97000002861023','20160518123900.00'
1023,678,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221227_until_kl1215_n50_100GB/ASPTCTX0001_npr735028517794-20180928/PT-20180928-103145-UAS-WB-FDG-3D-CT-VENFAS-WB-FDG-AC-3.2700mm,/media/andres/T7 Shield/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221227_until_kl1215_n50_100GB,ASPTCTX0001_npr735028517794-20180928,PT-20180928-103145-UAS-WB-FDG-3D-CT-VENFAS-WB-FDG-AC-3.2700mm,ASPTCTX0001,npr735028517794,20180928,543,1.2.840.113619.2.131.4240671514.1538123862.293064,npr735028517794,,'98.0','1.65',128,128,247,3.90625,3.90625,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, -698.90002441406]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.984.1537767585.268.12570.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20180928','103145','PET FDG WB 1 h','WB-FDG-AC','UAS-WB-FDG-3D CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','092300.00','392501664.0','6588.0','0.97000002861023','20180928092300.00'


In [9]:
master_data_imgszdist = master_data.groupby(['modality', 'image_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_imgszdist.to_excel(os.path.join(source_path, "master_data_imgszdist.xlsx"))
print(master_data_imgszdist.shape)
master_data_imgszdist.head(10)

(134, 3)


,modality,image_size,dicom_img
0,PT,"(128, 128, 247)",168
1,PT,"(256, 256, 287)",109
2,PT,"(256, 256, 314)",100
3,CT,"(512, 512, 1151)",98
4,CT,"(512, 512, 1409)",79
5,CT,"(512, 512, 1283)",75
6,PT,"(256, 256, 341)",65
7,PT,"(128, 128, 287)",59
8,CT,"(512, 512, 215)",57
9,CT,"(512, 512, 1523)",31


In [10]:
master_data_voxszdist = master_data.groupby(['modality', 'voxel_size']).agg({'dicom_img':'count'}).sort_values('dicom_img' ,ascending=False).reset_index()
master_data_voxszdist.to_excel(os.path.join(source_path, "master_data_voxszdist.xlsx"))
print(master_data_voxszdist.shape)
master_data_voxszdist.head(10)

(179, 3)


,modality,voxel_size,dicom_img
0,PT,"(3.90625, 3.90625, 3.27)",288
1,PT,"(1.953125, 1.953125, 2.79)",205
2,CT,"(0.976562, 0.976562, 0.625)",199
3,PT,"(1.953125, 1.953125, 2.8)",156
4,CT,"(0.976562, 0.976562, 3.75)",71
5,CT,"(0.976562, 0.976562, 0.949219)",48
6,PT,"(5.46875, 5.46875, 3.27)",23
7,CT,"(1.367188, 1.367188, 3.75)",21
8,CT,"(0.703125, 0.703125, 0.949219)",13
9,CT,"(0.916016, 0.916016, 0.949219)",6


In [11]:
master_data.columns

Index(['Unnamed: 0_x', 'directory', 'source_directory', 'patient_directory',
       'PET-CT_info', 'system', 'npr', 'scan_date', 'Unnamed: 0_y',
       'dicom_img', 'patient_id', 'patient_age', 'patient_weight',
       'patient_size', 'imgsz_x', 'imgsz_y', 'num_slices', 'voxsz_x',
       'voxsz_y', 'slice_thickness', 'image_size', 'voxel_size', 'img_pos',
       'img_orient', 'for_uid', 'att_corr', 'recons_method', 'image_type',
       'aquisition_dt', 'aquisition_time', 'study_desc', 'series_desc',
       'protocol', 'corr_img', 'modality', 'manufacturer',
       'manufacturer_model', 'radiopharmaceutical',
       'radiopharmaceutical_volume', 'radiopharmaceutical_start_time',
       'radionuclide_total_dose', 'radionuclide_half_life',
       'radionuclide_positron_fraction',
       'radiopharmaceutical_start_date_time'],
      dtype='object')

In [12]:
master_data = pd.read_excel(os.path.join(source_path, "Finalized_dataset.xlsx"))

#master_data['modality'] = master_data['modality'].apply(lambda x: x.strip().replace("'","").strip())
#master_data.to_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/Selected_exams_from_U-CAN-Lymfom_A_MasterDataset_2Oct2023.csv', index=False)


##### Summary for CT images

In [13]:
print("Mode of x image dims: ", master_data[master_data['modality']=='CT']['imgsz_x'].mode()[0])
print("Mode of y image dims: ", master_data[master_data['modality']=='CT']['imgsz_y'].mode()[0])
print("Mode of z image dims: ", master_data[master_data['modality']=='CT']['num_slices'].mode()[0])
print("Mode of x voxel dims: ", master_data[master_data['modality']=='CT']['voxsz_x'].mode()[0])
print("Mode of y voxel dims: ", master_data[master_data['modality']=='CT']['voxsz_y'].mode()[0])
print("Mode of z voxel dims: ", master_data[master_data['modality']=='CT']['slice_thickness'].mode()[0])

Mode of x image dims:  512
Mode of y image dims:  512
Mode of z image dims:  1151
Mode of x voxel dims:  0.976562
Mode of y voxel dims:  0.976562
Mode of z voxel dims:  0.625


In [14]:
master_data[master_data['modality']=='CT'][['imgsz_x','imgsz_y','num_slices','voxsz_x','voxsz_y','slice_thickness']].describe()

,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness
count,668.0,668.0,668.000000,668.000000,668.000000,668.000000
mean,512.0,512.0,1118.678144,0.944117,0.944117,1.252833
std,0.0,0.0,444.020499,0.112193,0.112193,1.117389
min,512.0,512.0,180.000000,0.683594,0.683594,0.625000
25%,512.0,512.0,1151.000000,0.884766,0.884766,0.625000
50%,512.0,512.0,1283.000000,0.976562,0.976562,0.625000
75%,512.0,512.0,1409.000000,0.976562,0.976562,0.949219
max,512.0,512.0,1991.000000,1.603750,1.603750,3.750000


##### Summary of PET images

In [15]:
print("Mode of x image dims: ", master_data[master_data['modality']=='PT']['imgsz_x'].mode()[0])
print("Mode of y image dims: ", master_data[master_data['modality']=='PT']['imgsz_y'].mode()[0])
print("Mode of z image dims: ", master_data[master_data['modality']=='PT']['num_slices'].mode()[0])
print("Mode of x voxel dims: ", master_data[master_data['modality']=='PT']['voxsz_x'].mode()[0])
print("Mode of y voxel dims: ", master_data[master_data['modality']=='PT']['voxsz_y'].mode()[0])
print("Mode of z voxel dims: ", master_data[master_data['modality']=='PT']['slice_thickness'].mode()[0])

Mode of x image dims:  256
Mode of y image dims:  256
Mode of z image dims:  247
Mode of x voxel dims:  1.953125
Mode of y voxel dims:  1.953125
Mode of z voxel dims:  3.27


In [16]:
master_data[master_data['modality']=='PT'][['imgsz_x','imgsz_y','num_slices','voxsz_x','voxsz_y','slice_thickness']].describe()

,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness
count,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000
mean,196.761905,196.761905,285.833333,2.910505,2.910505,3.014464
std,63.870140,63.870140,39.850117,1.069120,1.069120,0.237385
min,128.000000,128.000000,89.000000,1.953125,1.953125,2.790000
25%,128.000000,128.000000,247.000000,1.953125,1.953125,2.790000
50%,256.000000,256.000000,287.000000,1.953125,1.953125,2.800000
75%,256.000000,256.000000,314.000000,3.906250,3.906250,3.270000
max,256.000000,256.000000,449.000000,5.468750,5.468750,3.270000


### New size and spacing based on master data

new_size = [384, 384, 384]
new_spacing = [2, 2, 3]

In [17]:
master_data.head(2)
master_data.npr.nunique()

463

In [18]:
master_data.patient_directory.nunique()

668